In [1]:
import pandas as pd
import numpy as np
import math
import os

In [2]:
path = os.path.abspath(os.getcwd())
df = pd.read_csv(path + "/Data4/Ethnic Groups - Yearly.csv")
df2 = pd.read_csv(path + "/Data4/Ethnic Groups - Range.csv")

# drop a row in df and df2 if all columns are null
df = df.dropna(axis = 0, how = 'all')
df.drop_duplicates(inplace = True)
df2 = df2.dropna(axis = 0, how = 'all')
df2.drop_duplicates(inplace = True)
print(df.columns)
print(df2.columns)

Index(['村志代码 Gazetteer Code', '村志书名 Gazetteer Title', '民族 Ethnic Groups',
       'Division1', '1949', '1950', '1951', '1952', '1953', '1954', '1955',
       '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019'],
      dtype='object')
Index(['村志代码 Gazetteer Code', '村志书名 Gazetteer Title', '民族 Ethnic Groups',
       'Division1', 'Start Year', 'End Year', 'Data'],
      dtype='object')


In [3]:
categories = ["汉族 Han", "壮族 Zhuang" ,"回族 Hui", "满族 Manchu","维吾尔族 Uyghur","苗族 Miao","彝族 Yi","土家族 Tujia","藏族 Tibetan","蒙古族 Mongolian","侗族 Dong","布依族 Bouyei"
,"瑶族 Yao","白族 Bai","朝鲜族 Korean/Choson","哈尼族 Ho /Hani","黎族 Li","哈萨克族 Kazakh","傣族 Dai","畲族 She","傈僳族 Lisu","东乡族 Dongxiang","仡佬族 Gelao"
,"拉祜族 Lahu","佤族 Wa","水族 Sui","纳西族 Nakhi/Naxi","羌族 Qiang","土族 Tu","仫佬族 Mulao","锡伯族 Xibe","柯尔克孜族 Kyrgyz","景颇族 Jingpo","达斡尔族 Daur"
,"撒拉族 Salar","布朗族 Blang","毛南族 Maonan","塔吉克族 Tajik","普米族 Pumi","阿昌族 Achang","怒族 Nu","鄂温克族 Evenki/Ewenki" ,"京族 Gin","基诺族 Jino"
,"德昂族 De'ang/Deang","保安族 Bonan","俄罗斯族 Russian" ,"裕固族 Yugur","乌兹别克族 Uzbek" ,"门巴族 Monba","鄂伦春族 Oroqen" ,"独龙族 Dereng","赫哲族 Hezhen"
,"高山族 Gaoshan" ,"珞巴族 Lhoba","塔塔尔族 Tatar", "少数民族 (总) Ethnic Minorities (total)"]


In [4]:
def unionTwoLists(list1, list2):
    for category in list1:
        if category not in list2:
            list2.append(category)
    return list2

# create ethnic(category column) id mapping
count = 1
category_id = {}
for category in categories:
    if category not in category_id:
        category_id[category] = count
        count = count + 1
        
# create unit(division column) mapping
year_division1 = [unit for unit in df['Division1'].astype('category').unique()]
range_division1 = [unit for unit in df2['Division1'].astype('category').unique()]
division1 = unionTwoLists(year_division1, range_division1)

In [5]:
# in yearly and range df relpace ethnic with id
for category in category_id:
    df = df.replace(category, category_id[category])
    df2 = df2.replace(category, category_id[category])

In [6]:
# check replace results
df_replaced_category = [category for category in df['民族 Ethnic Groups'].astype('category').unique()]
df2_replaced_category = [category for category in df2['民族 Ethnic Groups'].astype('category').unique()]
print(df_replaced_category)
print(df2_replaced_category)

[57, 1, 2, 13, 30, 4, 10, 6, 16, 3, 12, 14, 11, 8, 7, 17, 15, 29, 9, 31, 19, 37, 33, 27, 21, 24, 25, 26, 39, 36, 34, 28, 42, 35, 52, 20, 41, 40, 53, 5, 23, 51, 45, 43, 18]
[1, 3, 4, 6, 15, 16, 12, 11, 2, 7, 8, 57, 10]


In [24]:
# create ethnic_df
ethnic_df = pd.DataFrame(columns = ['gazetteerId', 'categoryId', 'startYear', 'endYear', 'unitId', 'data'])
years = [str(i) for i in range(1949,2020)]
dic_for_ethnic_df = {'gazetteerId':[], 'categoryId':[], 'startYear':[], 'endYear':[], 'unitId':[], 'data':[]}

# Process yearly data
for i in range(len(df)):# each row
    for year in years: # 1949 - 2019
#         if df.iloc[i][year] != "NULL":
        dic_for_ethnic_df['gazetteerId'].append(df.iloc[i]['村志代码 Gazetteer Code'])
        dic_for_ethnic_df['categoryId'].append(df.iloc[i]['民族 Ethnic Groups'])
        dic_for_ethnic_df['unitId'].append(df.iloc[i]['Division1'])
        dic_for_ethnic_df['startYear'].append(int(year))
        dic_for_ethnic_df['endYear'].append(int(year))
        dic_for_ethnic_df['data'].append(df.iloc[i][year])

# Process range data
for i in range(len(df2)):
    dic_for_ethnic_df['gazetteerId'].append(df2.iloc[i]['村志代码 Gazetteer Code'])
    dic_for_ethnic_df['categoryId'].append(df2.iloc[i]['民族 Ethnic Groups'])
    dic_for_ethnic_df['unitId'].append(df2.iloc[i]['Division1'])
    dic_for_ethnic_df['startYear'].append(df2.iloc[i]['Start Year'])
    dic_for_ethnic_df['endYear'].append(df2.iloc[i]['End Year'])
    dic_for_ethnic_df['data'].append(df2.iloc[i]['Data'])


for attribute in ethnic_df.columns:
    ethnic_df[attribute] = dic_for_ethnic_df[attribute]

In [25]:
ethnic_df = ethnic_df.where(ethnic_df.notnull(), "NULL")
ethnic_df.head()

,gazetteerId,categoryId,startYear,endYear,unitId,data
0,3,57,1949,1949,人数 Number of People,NULL
1,3,57,1950,1950,人数 Number of People,NULL
2,3,57,1951,1951,人数 Number of People,NULL
3,3,57,1952,1952,人数 Number of People,NULL
4,3,57,1953,1953,人数 Number of People,NULL


In [27]:
# create dict "unit_id" stores {unit: id}
unit_id = {}
count = 1

for unit in ethnic_df['unitId'].astype('category').unique():
    if unit not in unit_id and unit:
        unit_id[unit] = count
        count = count + 1

dic_unit_df = {'id':[], 'name':[]}
for unit in unit_id:
    dic_unit_df['id'].append(unit_id[unit])
    dic_unit_df['name'].append(unit)
    
unit_df = pd.DataFrame(columns = ['id', 'name'])  
for attribute in unit_df.columns:
    unit_df[attribute] = dic_unit_df[attribute]
    
# replace unit with unit id
for unit in unit_id:
    ethnic_df = ethnic_df.replace(unit, unit_id[unit])

In [28]:
ethnic_df.head()

,gazetteerId,categoryId,startYear,endYear,unitId,data
0,3,57,1949,1949,1,3.0
1,3,57,1950,1950,1,3.0
2,3,57,1951,1951,1,3.0
3,3,57,1952,1952,1,3.0
4,3,57,1953,1953,1,3.0


In [29]:
# create ethnicCategory_df
category_df = pd.DataFrame(columns = ['id', 'name', 'parentId'])
dic_category_df = {'id':[], 'name':[], 'parentId':[]}

for category in category_id:
    dic_category_df['name'].append(category)
    dic_category_df['id'].append(category_id[category])
    dic_category_df['parentId'].append("NULL")
        
for attribute in category_df.columns:
    category_df[attribute] = dic_category_df[attribute]

In [30]:
# debug for foreign key constrain on gazetteerId
ethnic_df[ethnic_df['gazetteerId'] == 657]

,gazetteerId,categoryId,startYear,endYear,unitId,data


In [31]:
ethnic_df.to_csv('ethnic_groups_df2.csv', index = False, na_rep = "NULL")

In [41]:
category_df.to_csv('ethnic_group_category_df2.csv', index = False, na_rep = "NULL")

In [34]:
unit_df.to_csv('ethnic_group_unit_df2.csv', index = False, na_rep = "NULL")

In [33]:
unit_df

,id,name
0,1,人数 Number of People
1,2,百分比 Percentage (%)
2,3,NULL
3,4,户数 Number of Households
